In [4]:
import pandas as pd
import numpy as np
import datetime
import logging
from selenium.webdriver.remote.remote_connection import LOGGER as serverLogger
serverLogger.setLevel(logging.WARNING)
logging.getLogger("yahoo_oauth").setLevel(level=logging.INFO)

from csh_fantasy_bot import bot, nhl
from csh_fantasy_bot.roster_change_optimizer import RosterChangeSet
from csh_fantasy_bot.nhl import score_team



In [72]:
# true value ignores actuals if they exist and works off projections only
simulation_mode=False
week_number = 4
league_id = '403.l.41177'
# league_id = "403.l.18782"
manager = bot.ManagerBot(week_number, league_id=league_id, simulation_mode=simulation_mode)
league_scorer = manager.score_comparer
manager.opp_team_name

'Belfast Bruisers'

In [88]:
# paste roster change text into here
"""
Here is example of what roster change text should look like
It is output from the rosterchangeset.pretty_print method

Date: 2021-02-08, in: Alexis Lafreniere(8641), out: Teuvo Teravainen(5698)
Date: 2021-02-09, in: Zdeno Chara(1700), out: Kailer Yamamoto(7537)
Date: 2021-02-10, in: Adam Fox(7174), out: Evander Kane(4684)
"""
roster_change_text="""
Date: 2021-02-09, in: Ryan McDonagh(4251), out: Conor Garland(6865)
Date: 2021-02-09, in: Adam Fox(7174), out: Kailer Yamamoto(7537)
"""

roster_changes = RosterChangeSet.from_pretty_print_text(roster_change_text, manager.all_player_predictions)
scores_with_changes = manager.score_team(roster_change_set=roster_changes)[1]
scoring_result = league_scorer.score(scores_with_changes)[manager.stat_categories]
print(scoring_result)
print(f"\nScore: opp->({scoring_result.loc['score_opp'].sum()}), league->({scoring_result.loc['score_league'].sum()})") 
if (roster_change_text != ''):
    scores_ignore_roster_changes = manager.score_team()[1]
    score_result_no_changes = league_scorer.score(scores_ignore_roster_changes)
    print (f"\nWithout roster changes: opp->({score_result_no_changes.loc['score_opp'].sum()}), league->({score_result_no_changes.loc['score_league'].sum()})") 

G       A    +/-     PIM      SOG       FW     HIT
my-scores          13.310  20.140  4.030  36.430  126.630   58.230  78.730
opponent           12.870  19.060  5.030  16.990  118.820  118.820  42.690
difference_opp      0.440   1.080 -1.000  19.440    7.810  -60.590  36.040
mean-league        11.399  19.344  4.055  20.881  101.676   92.089  46.586
std dev             1.637   3.039  1.606   5.584   14.776   26.638  15.148
mean-opp           13.090  19.600  4.530  26.710  122.725   88.525  60.710
num_stds_opp        0.269   0.355 -0.623   3.481    0.529   -2.275   2.379
score_opp           0.269   0.355 -0.623   1.500    0.529   -1.500   1.500
win_loss_opp        1.000   1.000 -1.000   1.000    1.000   -1.000   1.000
difference_league   1.911   0.796 -0.025  15.549   24.954  -33.859  32.144
score_league        1.167   0.262 -0.016   1.500    1.500   -1.271   1.500

Score: opp->(2.03), league->(4.642)

Without roster changes: opp->(-1.222), league->(0.9720000000000001)


In [89]:
scores_ignore_roster_changes = scores_with_changes
games_played = scores_ignore_roster_changes['G'].groupby(level=1).count()
result = manager.all_player_predictions.copy()
result['GP'] = games_played
result[result.GP.notna()].sort_values('GP', ascending=False).round(2)[['name', 'GP', 'fpts']]

,name,GP,fpts
player_id,,,
5696,Tom Wilson,4.0,1.15
5712,Jonathan Marchessault,4.0,1.02
7902,Brady Tkachuk,4.0,0.96
3358,Brent Burns,3.0,0.43
4684,Evander Kane,3.0,1.39
5697,Tomas Hertl,3.0,0.46
3637,Alex Ovechkin,3.0,0.72
4251,Ryan McDonagh,3.0,0.51
5986,Darnell Nurse,3.0,0.47
